## Image classification of Dog and Cat using Transfer Learning

when we have a relative small Dataset, a Transfer Learning is a super-effective technique. It's consist of using a pre-trained model.
This model has been Trained on an extremely large dataset, and we would be able to transfer weights which were learned through
hundreds of hours of training on multiple high powered GPUs. 

We are using in this project the **Inception-v3** model

### Let's import the necessary Packages

In [ ]:
import os 
import tensorflow as tf

from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras_preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import RMSprop, Adam, Adagrad

import numpy as np

import matplotlib.pyplot as plt

Remember you must download the dataset. You can found it in Kaggle(Dogs Cats image classification Dataset). 
I done it and save in my local computer space

In [ ]:
## Define the Dataset path and Train_set and Test_set directory

db_path = "C:/Users/nguim/OneDrive/Bureau/Learning/Project ML/Datasets/dogs_cats_dataset"

train_dir = os.path.join(db_path, 'train')
test_dir = os.path.join(db_path, 'test')

In [ ]:
print(type(train_dir))
print(type(test_dir))

### Image Preprocessing and Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    rescale=1./255, 
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=35, 
    zoom_range=0.3,
    horizontal_flip=True, 
    data_format='channels_last'
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    rescale=1./255,
    shear_range=0.2, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    rotation_range=35, 
    zoom_range=0.3, 
    horizontal_flip=True, 
    data_format='channels_last'
)

In [ ]:
train_set_generate = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(299, 299), 
    batch_size=32, 
    class_mode='binary'
)


test_set_generate = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(299, 299), 
    batch_size=32, 
    class_mode='binary'
)

In [ ]:
print(type(train_set_generate))
print(type(test_set_generate))

### Now let's plot the first nine images and labels in the train_set_generate

In [ ]:
# Get the first batch of images and labels
images, labels = next(train_set_generate)

# Plot the first nine images and their labels
fig, axs = plt.subplots(3, 3, figsize=(4, 4))
for i in range(9):
    ax = axs[i//3, i%3]
    ax.imshow(images[i] / 255)
    ax.set_title(f"Label: {labels[i]}")
    ax.axis('off')
plt.show()

the first nine image with theirs label looks all black(not on their original color). This ay due to the fact that we used the
"preprocessing_function" in the ImageDataGenerator

### Now let's define the base model or the pre_trained_modelof Transfer learning

In [ ]:
image_shape = (299, 299, 3)

pre_trained_model = InceptionV3(
    input_shape=image_shape, 
    include_top=False,  # leave out the last fully connected layer
    weights='imagenet'
)

In [ ]:
# Let's make all the layer of pre_trained_model non trainable. 
# That means we freeze the models parameters

pre_trained_model.trainable = False

In [ ]:
# Let's see the summary of base model

pre_trained_model.summary()

### Now let's build our principal Model 

Our model is going to use the pre_trained_model as a root. 
We use binary_crossentropy as the loss metric as we have 2 target classes. 
Our Optimizer is RMSprop with learning rate of 0.001(you can experiment wit Adam or Adagrad optimizer
this will also work well)

In [ ]:
# Flatten the output layer of pre_trained_model to 1 dimension
x = Flatten()(pre_trained_model.output)

# Let's add a fully connected layer with 102 hidden init and ReLU activation function 
x = Dense(1024, activation='relu')(x)

# Let's add the Dropout of rate 0.2
x = Dropout(0.2)(x)

# Now let's add our final layer for classification
x = Dense(1, activation='sigmoid')(x)

In [ ]:
# Let's define now our model base on pre_trained_model
model = Model(pre_trained_model.input, x)

model.summary()

In [ ]:
# Before we train our model, let compile it
model.compile(
    optimizer= RMSprop(learning_rate=0.001), 
    loss='binary_crossentropy', 
    metrics= ['acc']
)

In [ ]:
# Let's train our model
callbacks = myCallback()
model.fit_generator(
    train_set_generate, 
    validation_data= test_set_generate, 
    epochs=5, 
    steps_per_epoch=5, 
    validation_steps=3, 
    verbose=2, 
    callbacks=[callbacks]
)